### Aho-Corasick 알고리즘
- 문자열 $S$ 와 문자열 집합 $P$ 가 주어질 때, $P$ 의 각 문자열이 $S$ 과 얼마나 매칭되는지를 찾는 알고리즘
  - 일대다 패턴 매칭 알고리즘이다.
- Trie에 KMP 개념을 집어넣은 원리로 작동한다.
- $\displaystyle\Omicron(|S| + \sum_{p \in P} |p|)$ 의 시간복잡도를 가진다.

### 동작
1. $P$ 에 해당하는 문자열들에 대해 Trie를 구성한다. 그러한 Trie를 $G$ 라고 하자
    - 이때 $p$ 의 마지막 문자가 가리키는 노드에 체크(output check)를 해두어 `이 노드에 도착했다면 문자열 한 개와 매칭됐다`는 것을 표시한다.
2. Trie에서 Failure function(Failure link, Suffix link라고도 불린다)을 계산한다.
    - $p$ 의 Failure function $f$ 는 $p$ 의 접미사 중 $P$ 에 속하는 문자열의 최대 길이를 가진 접미사이다.\
    루트 노드 $r$ 이 아닌 각 노드 $v \in G$ 에 대해서 $r \to w$ 로 가는 경로가, $r \to v$ 로 가는 경로의 접미사이며, $w \ne v$ 이면서 그러한 것이 여러개 있다면 그 중 가장 긴 것.
      - ![Failure function 1](assets/ahocorasick-2.png) \
      예시1. $f(ada) = a, f(adab) = ab$, $f(adad) = ad$ 이다.
      - ![Failure function 2](assets/ahocorasick-1.png) \
      예시2. 가는 경로가 여러개 있다면 가장 긴 것을 선택한다. 따라서 $f(adada) = ada$ 이다.
        - 현재 위치에서 깊이가 가장 적게 내려가는 경로가 된다
    - BFS를 사용하여 깊이가 작은 노드부터 방문하면 $f$ 를 DP적용 하듯이 결정할 수 있다.
3. Failure function이 계산된 트라이에서 $S$ 와 매칭하는 문자열을 찾는다.
    - output check가 되어있는 노드에 도달하면 매칭하는 문자열 하나를 찾은 것이다.

### 특징
- $f(x) = y$ 일 때 $y$ 는 $x$ 의 접미사이다.
- 어떤 트라이상의 문자열 $x, yx$ 가 있고, $f(yx) = x$ 라고 하자. 여기에 문자열 $z$ 를 붙여서 $xz, yxz \in W$ 라고할 때, $f(yxz) = xz$ 이다.
  - 따라서 어떤 문자열 $T$ 가 있을 때, 문자 $a$ 를 이어붙이면 $f(T \oplus a) = f(T) \oplus a$ 이다. 

### 구현
- 주석은 9250(문자열 집합 판별) 참고
- 매칭의 여부를 확인한다면 __contains__를, 매칭될 문자열의 개수를 찾는다면 query를 사용.
  - 모든 문자열을 출력하는 문제도 있을 것 같다

In [ ]:
C_SIZE = 26
C_TO_I = lambda x: x - 97
class Trie:
  def __init__(self) :
    self.go = [None] * C_SIZE
    self.fail = None
    self.output = False

  def add(self, k) :
    if not k :
      self.output = True
      return

    v = C_TO_I(k[0])
    if self.go[v] is None :
      self.go[v] = Trie()
    self.go[v].add(k[1:])

from collections import deque 
class AhoCorasick:
  def __init__(self, trie: Trie):
    self.trie = trie
    self.trie.fail = self.trie
    
    Q = deque([self.trie]) 
    while Q :
      u = Q.popleft()
      for i in range(C_SIZE) :
        v = u.go[i]
        if not v: continue
        
        if u == self.trie :
          v.fail = self.trie
        else :
          w = u.fail
          while w != self.trie and not w.go[i] :
            w = w.fail
          if w.go[i] :
            w = w.go[i]
          v.fail = w
        v.output |= v.fail.output
        Q.append(v)
      
  def __contains__(self, s: str):
    u = self.trie
    for c in s :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      if u.output :
        return True
    return False

  def query(self, s: str):
    u = self.trie
    cnt = 0 
    for c in s :
      i = C_TO_I(c)
      while u != self.trie and not u.go[i] :
        u = u.fail
      if u.go[i] :
        u = u.go[i]
      cnt += u.output
    return cnt